In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [32]:
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil
import pandas as pd

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20, viridis
from bokeh.layouts import gridplot, column, layout, grid, row
from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [3]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [183]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/MOVIMPU.TPS"
df = parser(path);df

,cod_imput,tipo_movim,fecha_comprob,periodo_contable,num_comprob,descripcion,importe,actividad,tipo_proveedor,cod_provincia,nro_ordenpago,memo
0,2,Orden De Pago,2013-12-15,2013-12-15,1725,Luciano R. Retamoso,13292.30,0,15,5,1725,NaN
1,2,Orden De Pago,2014-01-20,2014-01-20,1728,Luciano R. Retamoso,8306.80,0,15,5,1728,NaN
2,105,Cobranza,2014-11-25,2014-11-25,231,Comercial Eusebia S.A.,82458.52,0,27,13,231,NaN
3,120,Documento De Contado,2014-11-25,2014-11-25,225,A.F.I.P,996.00,0,0,0,0,NaN
4,118,Documento De Contado,2014-11-21,2014-11-21,226,A.F.I.P,14657.09,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9638,2,Orden De Pago,2016-02-09,2016-02-09,3372,Total Especialidades Arg S.A.,2794.53,0,22,168,3372,NaN
9639,42,Facturas Cuenta Corriente,2015-09-15,2015-09-30,3,Alicia Bengochea,12922.50,0,1,184,0,NaN
9640,42,Facturas Cuenta Corriente,2015-10-15,2015-10-31,4,Alicia Bengochea,13050.00,0,1,184,0,NaN
9641,42,Facturas Cuenta Corriente,2015-11-15,2015-11-30,5,Alicia Bengochea,10740.00,0,1,184,0,NaN


In [182]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/ACUMULA.TPS"
df = parser(path);df

,codigo,descripcion,compra_cta_cte,compra_cont,nota_credito_recibida,nota_debito_recibida,egresos_cta_cte,venta_cta_cte,nota_debito_emitida,cobranzas,unid_generadora,cuenta_gastos
0,22,Sueldos,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4,1
1,23,Gastos Varios,8787.65,6250.27,0.00,0.00,0.00,0.00,45407.60,0.00,1,1
2,24,Seguros,53794.07,6673.91,0.00,0.00,0.00,0.00,0.00,0.00,1,1
3,25,Ventas Animales,93628.49,0.00,0.00,0.00,0.00,3146445.92,9695.00,0.00,4,1
4,26,Venta Leche,0.00,0.00,0.00,0.00,0.00,7289610.83,0.00,0.00,6,1
5,27,Venta De Granos,65803.47,0.00,0.00,0.00,0.00,2070067.02,0.00,0.00,5,0
6,28,Compra De Granos,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,1
7,29,Compra Iva,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,1
8,30,Intereses,981.50,0.00,0.00,7457.38,0.00,0.00,0.00,351.34,0,1
9,31,Creditos,1546130.11,0.00,0.00,0.00,0.00,0.00,425548.20,446200.00,8,1


In [185]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/IMPUTACI.TPS"
df = parser(path);df

,codigo,descripcion,compra_cta_cte,compra_contado,nota_credito_recibida,nota_debito_recibida,egresos_cta_cte,egresos_contado,venta_cta_cte,cobranzas,unid_generadora,cuenta_gastos,donde_acumula,total_egresos,porcentaje
0,131,Imp Ley 25413 Afip,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
1,132,Comision Transferencia,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
2,133,Franqueo,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
3,134,Comision Mant. De Cuenta,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
4,135,Comision Resumen,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
5,136,Comision Canje O/Bancos,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
6,137,Comision Gastos Chequera,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1,19,0.00,0.00
7,138,Intereses Bancarios,4359.25,2416.10,0.00,0.00,0.00,0.00,0.00,0.00,0,1,30,0.00,0.00
8,140,Semilla De Maiz,122621.22,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,1,1,70695.00,1.22
9,141,Servicios De Saco De Grano,8062.40,0.00,0.00,13049.40,0.00,0.00,0.00,0.00,5,1,15,33862.83,0.58


In [4]:
from app.models import Movimpu, Actividad, Acumula
from bokeh.transform import cumsum
from math import pi

In [85]:
mov_impu = Movimpu.objects.values(
    'cod_imput__descripcion',
    'cod_imput__donde_acumula__descripcion',
    'periodo_contable',
    'tipo_movim',
    'importe',
    'actividad__descripcion')

df = DataFrame(mov_impu)
df.rename(columns={
    'cod_imput__descripcion': 'imputacion',
    'actividad__descripcion': 'actividad',
    'cod_imput__donde_acumula__descripcion': 'imputacion_acumulada'
}, inplace=True)
df['tipo'] = 'Egresos'
df.loc[(df.tipo_movim == 'Cobranza') | (df.tipo_movim == 'Nota De Crédito'), 'tipo'] = 'Ingresos'

In [86]:
df

,imputacion,imputacion_acumulada,periodo_contable,tipo_movim,importe,actividad,tipo
0,Pago Proveedores,Administracion,2013-12-15,Orden De Pago,13292.30,None,Egresos
1,Pago Proveedores,Administracion,2014-01-20,Orden De Pago,8306.80,None,Egresos
2,Deudores Por Ventas,Administracion,2014-11-25,Cobranza,82458.50,None,Ingresos
3,Volante Pago Trab. De Casas,Impuestos,2014-11-25,Documento De Contado,996.00,None,Egresos
4,Retenciones Y Percepciones,Impuestos,2014-11-21,Documento De Contado,14657.10,None,Egresos
...,...,...,...,...,...,...,...
9598,Pago Proveedores,Administracion,2016-02-09,Orden De Pago,2794.53,None,Egresos
9599,Alquiler Campo Ganaderia,Alquiler Rural,2015-09-30,Facturas Cuenta Corriente,12922.50,None,Egresos
9600,Alquiler Campo Ganaderia,Alquiler Rural,2015-10-31,Facturas Cuenta Corriente,13050.00,None,Egresos
9601,Alquiler Campo Ganaderia,Alquiler Rural,2015-11-30,Facturas Cuenta Corriente,10740.00,None,Egresos


In [97]:
df = df.append({'actividad': 'Nueva', 
           'importe': 15000,
           'tipo': 'Ingreso',
           'imputacion_acumulada': 'impu ac', 
           'imputacion': 'impu'}, ignore_index=True)

In [99]:
gb = DataFrame(df.groupby(['actividad', 'tipo', 'imputacion_acumulada'], as_index=False).importe.sum())
gb['Egresos'] = gb.loc[gb.tipo == 'Egresos'].importe
gb['% Egresos'] = gb.Egresos / gb.Egresos.sum() * 100
gb['Ingresos'] = gb.loc[gb.tipo == 'Ingresos'].importe
gb['% Ingresos'] = gb.Ingresos / gb.Ingresos.sum() * 100
gb = gb.where((pd.notnull(gb)), None); gb

,actividad,tipo,imputacion_acumulada,importe,Egresos,% Egresos,Ingresos,% Ingresos
0,Agricultura,Egresos,Venta De Granos,384846,384846,4.20231,None,None
1,Creditos,Egresos,Creditos,42000,42000,0.458617,None,None
2,Ganaderia,Egresos,Ventas Animales,8.6978e+06,8.6978e+06,94.9752,None,None
3,Nueva,Ingreso,impu ac,15000,None,None,None,None
4,Tambo,Egresos,Servicios Agrop Tambo,33324,33324,0.36388,None,None


In [83]:
gb = DataFrame(df.groupby(['actividad', 'tipo', 'imputacion_acumulada', 'imputacion'], as_index=False).importe.sum())
gb['egresos_acumulados'] = gb.loc[gb.tipo == 'Egresos'].groupby(['actividad', 'imputacion_acumulada']).importe.transform('sum')
gb['% Egresos Acumulados'] = gb.egresos_acumulados / gb.egresos_acumulados.sum() * 100
gb['ingresos_acumulados'] = gb.loc[gb.tipo == 'Ingresos'].groupby(['actividad', 'imputacion_acumulada']).importe.transform('sum')
gb['% Ingresos Acumulados'] = gb.ingresos_acumulados / gb.ingresos_acumulados.sum() * 100

gb['egresos'] = gb.loc[gb.tipo == 'Egresos'].importe
gb['% Egresos'] = gb.egresos_acumulados / gb.egresos * 100
gb['ingresos'] = gb.loc[gb.tipo == 'Ingresos'].importe
gb['% Ingresos'] = gb.ingresos_acumulados / gb.ingresos_acumulados.sum() * 100

gb = gb.where((pd.notnull(gb)), None); gb

,actividad,tipo,imputacion_acumulada,imputacion,importe,egresos_acumulados,% Egresos Acumulados,ingresos_acumulados,% Ingresos Acumulados,egresos,% Egresos,ingresos,% Ingresos
0,Agricultura,Egresos,Venta De Granos,Venta De Granos,384846,384846,2.15531,None,None,384846,100,None,None
1,Creditos,Egresos,Creditos,Creditos,42000,42000,0.235218,None,None,42000,100,None,None
2,Ganaderia,Egresos,Ventas Animales,Gastos Varios Venta De Animale,286468,8.6978e+06,48.7114,None,None,286468,3036.22,None,None
3,Ganaderia,Egresos,Ventas Animales,Ventas Animales,8.41133e+06,8.6978e+06,48.7114,None,None,8.41133e+06,103.406,None,None
4,Tambo,Egresos,Servicios Agrop Tambo,Tambero,33324,33324,0.186629,None,None,33324,100,None,None


In [68]:
df = df[df.actividad == 'Ganaderia']; df

,imputacion,imputacion_acumulada,periodo_contable,tipo_movim,importe,actividad,tipo
1757,Ventas Animales,Ventas Animales,2014-02-28,Ventas Cuenta Corriente,8000.00,Ganaderia,Egresos
1847,Ventas Animales,Ventas Animales,2014-08-31,Ventas Cuenta Corriente,17113.10,Ganaderia,Egresos
1858,Ventas Animales,Ventas Animales,2014-03-31,Ventas Cuenta Corriente,97712.40,Ganaderia,Egresos
1913,Ventas Animales,Ventas Animales,2014-08-31,Ventas Cuenta Corriente,18000.00,Ganaderia,Egresos
1917,Ventas Animales,Ventas Animales,2014-08-31,Ventas Cuenta Corriente,20000.00,Ganaderia,Egresos
1919,Ventas Animales,Ventas Animales,2014-08-31,Ventas Cuenta Corriente,20000.00,Ganaderia,Egresos
2024,Ventas Animales,Ventas Animales,2014-09-30,Ventas Cuenta Corriente,40000.00,Ganaderia,Egresos
2099,Ventas Animales,Ventas Animales,2014-09-30,Ventas Cuenta Corriente,23000.00,Ganaderia,Egresos
2146,Ventas Animales,Ventas Animales,2014-10-31,Ventas Cuenta Corriente,75000.00,Ganaderia,Egresos
2170,Ventas Animales,Ventas Animales,2014-09-30,Ventas Cuenta Corriente,7571.13,Ganaderia,Egresos


In [82]:
df = df.append({'actividad': 'Agricultura', 
                'imputacion_acumulada': 'Ventas de otra cosa',
                'imputacion': 'Ventas de Granos',
                'tipo': 'Ingresos', 
                'importe': 200000}, ignore_index=True)
df_ingresos = DataFrame(df.groupby(['imputacion_acumulada', 'imputacion'], as_index=False).importe.sum())
# df_ingresos.loc[df_ingresos.tipo == 'Ingresos', 'color'] = 'green'
# df_ingresos.loc[df_ingresos.tipo == 'Egresos', 'color'] = 'red'
# df_ingresos.loc[df_ingresos.tipo == 'Egresos', 'porcentaje'] = df_ingresos[df_ingresos.tipo == 'Egresos'].importe / df_ingresos[df_ingresos.tipo == 'Egresos'].importe.sum() * 100
# df_ingresos.loc[df_ingresos.tipo == 'Ingresos', 'porcentaje'] = df_ingresos[df_ingresos.tipo == 'Ingresos'].importe / df_ingresos[df_ingresos.tipo == 'Ingresos'].importe.sum() * 100
# df_ingresos['porcentaje'] = round(df_ingresos.porcentaje, 2)
# df_ingresos.set_index(['imputacion_acumulada', 'imputacion'], inplace=True)
df_ingresos.sort_values('importe')

,imputacion_acumulada,imputacion,importe
3,Ventas de Granos,Ventas de Granos,200000.00
4,Ventas de otra cosa,Ventas de Granos,200000.00
0,Ventas Animales,Gastos Varios Venta De Animale,286468.12
2,Ventas de Granos,Ventas Granos,400000.00
1,Ventas Animales,Ventas Animales,8411331.25


In [86]:
plot_grid = []
for x in df_ingresos.imputacion_acumulada.unique():
    df_pie = df_ingresos[df_ingresos.imputacion_acumulada == x].copy()
    angle = df_pie.importe / df_pie.importe.sum() * 2 * pi
    df_pie['angle'] = round(angle, 2)
    df_pie.sort_values('importe', inplace=True)
    df_pie['color'] = viridis(len(df_pie['importe']))
    source = ColumnDataSource(df_pie)
    tooltips = [("Imputacion", "@imputacion"),
                ("Importe total", "@importe{$0.00}")]
    hovertool = HoverTool(names=['wedge'], tooltips=tooltips)
    plot1 = figure(
        # title='Comparación de gastos por categoría',
        tools=["tap,reset,save,crosshair", hovertool],
        toolbar_location='above',
        height=500,
        width=700,
        title=x
    )
    plot1.wedge(x=0, y=0, radius=0.5,
               start_angle=cumsum('angle', include_zero=True),
               end_angle=cumsum('angle'),
               line_color='white', line_width=0.5, fill_color='color',
               legend='imputacion',
               source=source, name='wedge')
    plot_grid.append(plot1)
    
show(grid(plot_grid))

In [45]:
angle = df_ingresos.importe / df_ingresos.importe.sum() * 2 * pi
df_ingresos['angle'] = round(angle, 2)
df_ingresos.sort_values('importe', inplace=True)
df_ingresos['color'] = viridis(len(df_ingresos['importe']))
source = ColumnDataSource(df_ingresos)
tooltips = [("Imputacion", "@imputacion_acumulada_imputacion"),
            ("Importe total", "@importe{$0.00}")]
hovertool = HoverTool(names=['wedge'], tooltips=tooltips)
plot1 = figure(
    # title='Comparación de gastos por categoría',
    tools=["tap,reset,save,crosshair", hovertool],
    toolbar_location='above',
    height=700,
    width=900
)
plot1.wedge(x=0, y=0, radius=0.5,
           start_angle=cumsum('angle', include_zero=True),
           end_angle=cumsum('angle'),
           line_color='white', line_width=0.5, fill_color='color',
           legend='imputacion_acumulada_imputacion',
           source=source, name='wedge')
plot2 = figure(
    # title='Comparación de gastos por categoría',
    tools=["tap,reset,save,crosshair", hovertool],
    toolbar_location='above',
    height=700,
    width=900,
    y_range=FactorRange(*source.data['imputacion_acumulada_imputacion'])
)
plot2.hbar(y='imputacion_acumulada_imputacion', height=0.9, right='importe', source=source, fill_color='color'
)

# plot.sizing_mode = 'fixed'

show(plot1)
show(plot2)